In [1]:
!pip install ../SeREGen

Processing /Users/NikhilKalidasu/Documents/DNA Visualization/SeREGen/SeREGen
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for SeREGen: filename=SeREGen-0.1.0-py3-none-any.whl size=48175 sha256=de3f324ad52dcbf4ca57be513df8cfb6ff091af2224e0ea87fa989fcd9b53953
  Stored in directory: /private/var/folders/_t/pfxf9f7x0nz1m6vckn9glz_40000gn/T/pip-ephem-wheel-cache-5alqzug1/wheels/c6/79/e5/431bc1fc7477a043b52c65c69a40862e34f24b12add06f2e4c
Successfully built SeREGen
  Attempting uninstall: SeREGen
    Found existing installation: SeREGen 0.1.0
    Uninstalling SeREGen-0.1.0:
      Successfully uninstalled SeREGen-0.1.0

[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import torch
torch.manual_seed(2021)
import pickle
with open('edit_qiita_large_np.pkl', 'rb') as f:
    sequences, distances = pickle.load(f)

In [3]:
dense_depth = 1
seq_len = sequences['train'].shape[-1]
compress_factor = 1
conv_filters = 64
conv_kernel_size=16
attn_heads = 2

In [4]:
from SeREGen.encoders import ModelBuilder
builder = ModelBuilder((sequences['train'].shape[-1],), input_dtype=torch.float32, float_type=torch.float32)
builder.one_hot_encoding(4)
# builder.transpose()
# builder.dense(seq_len)
# builder.dense(seq_len, depth=dense_depth, residual=True)
# builder.conv1D(conv_filters, conv_kernel_size, residual=True)
# # builder.attention(attn_heads, seq_len // compress_factor)
# builder.dense(seq_len, residual=True)
# builder.conv1D(16, 4, residual=True)

from SeREGen.comparative_encoder import ComparativeEncoder
from SeREGen.distance import EditDistance
dist = EditDistance(silence=True)
model = ComparativeEncoder.from_model_builder(builder, repr_size=2, embed_dist='hyperbolic', batch_size=128,
                                              norm_type='soft_clip', dist=dist)
model.summary()

/Users/NikhilKalidasu/miniforge3/envs/pytorch/lib/python3.10/site-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/NikhilKalidasu/miniforge3/envs/pytorch/lib/python3.10/site-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/Users/NikhilKalidasu/miniforge3/envs/pytorch/lib/python3.10/site-packages/torc

Layer (type:depth-idx)                   Param #
Sequential                               --
├─OneHotEncoding: 1-1                    --
├─Flatten: 1-2                           --
├─Linear: 1-3                            2
│    └─Sequential: 2-1                   --
│    │    └─Linear: 3-1                  1,218
Total params: 1,220
Trainable params: 1,220
Non-trainable params: 0

In [ ]:
import numpy as np
seqs = sequences['train']
distance_on = np.array(['A', 'C', 'G', 'T'])[seqs]
distance_on = np.fromiter((''.join(i) for i in distance_on), dtype=object).astype(str)
model.fit(seqs, loss='mse', distance_on=distance_on, epoch_factor=4, patience=3,
          suppress_grad_warn=['vanishing', 'nan'], clip_grad=False, encoding_loss="euclidean_dist",
          euclidean_dist_alpha=.1)

Epoch 1:


Training model (loss: 4.6192e-01): 100%|██████████████████████████████████████████████| 219/219 [00:05<00:00, 42.99it/s]


Epoch time: 5.9456987380981445
Epoch 2:


Training model (loss: 3.6338e-01): 100%|██████████████████████████████████████████████| 219/219 [00:04<00:00, 53.52it/s]


Epoch time: 4.248239755630493
Epoch 3:


...

Training model (loss: 6.7208e-02): 100%|██████████████████████████████████████████████| 219/219 [00:03<00:00, 55.92it/s]


Epoch time: 4.054839134216309
Epoch 12:


Training model (loss: 6.6891e-02): 100%|██████████████████████████████████████████████| 219/219 [00:04<00:00, 54.70it/s]


Epoch time: 4.146183252334595
Epoch 13:


Training model (loss: 6.7794e-02):  28%|█████████████▎                                 | 62/219 [00:01<00:02, 54.98it/s]

In [ ]:
# model.save('2d')

In [ ]:
# from SeREGen.comparative_encoder import ComparativeEncoder
# from SeREGen.distance import EditDistance
# dist = EditDistance()
# model = ComparativeEncoder.load('2d')

In [ ]:
print("Eval on train")
results = model.evaluate(seqs, distance_on, sample_size=1000)

In [ ]:
print("Eval on test")
distance_on_test = np.array(['A', 'C', 'G', 'T'])[sequences['test']]
distance_on_test = np.fromiter((''.join(i) for i in distance_on_test), dtype=object).astype(str)
results = model.evaluate(sequences['test'], distance_on_test, sample_size=1000)

In [ ]:
import torch
x, y = torch.from_numpy(results[0]).float(), torch.from_numpy(results[1]).float()
mx, my = torch.mean(x), torch.mean(y)
xm, ym = x - mx, y - my
r_num = torch.sum(xm * ym)
r_den = torch.sqrt(torch.sum(xm**2) * torch.sum(ym**2))
r = r_num / (r_den + 1e-8)  # Adding a small epsilon to avoid division by zero
r = torch.clamp(r, min=-1.0, max=1.0)
r

In [ ]:
np.mean((results[0] - results[1]) ** 2)

In [ ]:
np.mean((results[0] - results[1]) ** 2) ** .5

In [ ]:
from scipy.stats import pearsonr
pearsonr(*results).statistic ** 2

In [ ]:
np.mean(np.abs(results[1] - results[0]))

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(results[0], results[1], alpha=.1)
plt.show()

In [ ]:
encodings = model.transform(seqs)
import matplotlib.pyplot as plt
plt.scatter(encodings[:1000, 0], encodings[:1000, 1], alpha=.1)
plt.show()